# Export Genie Assets

Dynamic export of Genie Space assets for DTA metadata.

## Output Files
- **add_table_comments.sql**: Auto-generated table and column comments
- **genie_instructions.txt**: DTA-specific Genie space instructions
- **example_queries.sql**: Parameterized SQL queries for training Genie

## Output Path
```
/Volumes/{catalog}/bronze_md/{source_volume}/{source_root}/export/genie/
├── sql/
│   ├── add_table_comments.sql
│   └── example_queries.sql
├── genie_instructions.txt
└── export_manifest.json

Example: /Volumes/aira_test/bronze_md/clinical_data_standards/test/export/genie/
```


In [ ]:
# Cell 1: Get Configuration from Setup Task
import json
from datetime import datetime

# Get globals from setup task
try:
    setup_result = dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
    globals_dict = json.loads(setup_result) if isinstance(setup_result, str) else setup_result
    print(f"✓ Loaded globals from setup task")
except Exception as e:
    print(f"⚠ Could not load from setup task: {e}")
    globals_dict = {
        "catalog": "aira_test",
        "bronze_schema": "bronze_md",
        "silver_schema": "silver_md",
        "gold_schema": "gold_md"
    }

# Extract configuration
catalog = globals_dict.get("catalog", "aira_test")
bronze_schema = globals_dict.get("bronze_schema", "bronze_md")
silver_schema = globals_dict.get("silver_schema", "silver_md")
gold_schema = globals_dict.get("gold_schema", "gold_md")

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")


In [ ]:
# Cell 2: Get Job Parameters and Pipeline Config

# Widget parameters (passed from job)
# source_root is the FOLDER inside the volume (e.g., "test", "historical_data")
dbutils.widgets.text("source_root", "test", "Source Root")

source_root = dbutils.widgets.get("source_root")

# Get pipeline config to read source_volume (consistent with file_processor pattern)
try:
    pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
    source_volume = pipeline_config.get('source_volume', 'clinical_data_standards')
    print(f"✓ Loaded pipeline config from setup task")
except Exception as e:
    print(f"⚠ Could not load pipeline config: {e}")
    source_volume = 'clinical_data_standards'

print(f"="*60)
print(f"EXPORT GENIE ASSETS")
print(f"="*60)
print(f"Source Volume: {source_volume}")
print(f"Source Root (folder): {source_root}")
print(f"Catalog: {catalog}")
print(f"="*60)


In [ ]:
# Cell 3: Build Export Paths
import os

timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

# Build paths - consistent with file_processor pattern
# Path: /Volumes/{catalog}/{schema}/{volume}/{folder}/export/genie/
export_base = f"/Volumes/{catalog}/{bronze_schema}/{source_volume}/{source_root}/export/genie"
sql_dir = f"{export_base}/sql"

print(f"Export Base: {export_base}")
print(f"SQL Directory: {sql_dir}")

# Create directories
dbutils.fs.mkdirs(export_base.replace("/Volumes", "dbfs:/Volumes"))
dbutils.fs.mkdirs(sql_dir.replace("/Volumes", "dbfs:/Volumes"))
print(f"✓ Created export directories")


In [ ]:
# Cell 4: Discover Tables in Schemas

def get_tables_in_schema(schema_name):
    """Get list of tables in a schema."""
    try:
        tables_df = spark.sql(f"SHOW TABLES IN {catalog}.{schema_name}")
        tables = [row.tableName for row in tables_df.collect()]
        return tables
    except Exception as e:
        print(f"⚠ Could not list tables in {schema_name}: {e}")
        return []

# Discover tables in bronze and gold schemas
bronze_tables = get_tables_in_schema(bronze_schema)
gold_tables = get_tables_in_schema(gold_schema)

print(f"\n📦 Bronze Tables ({len(bronze_tables)}):")
for t in bronze_tables:
    print(f"  - {t}")

print(f"\n🥇 Gold Tables ({len(gold_tables)}):")
for t in gold_tables:
    print(f"  - {t}")


In [ ]:
# Cell 5: Get Column Metadata for Each Table

def get_table_metadata(schema_name, table_name):
    """Get table comment and column metadata."""
    full_table = f"{catalog}.{schema_name}.{table_name}"
    
    try:
        # Get table properties including comment
        props_df = spark.sql(f"DESCRIBE TABLE EXTENDED {full_table}")
        props = {row.col_name: row.data_type for row in props_df.collect()}
        
        # Extract table comment from properties
        table_comment = None
        for row in props_df.collect():
            if row.col_name == 'Comment' or row.col_name == 'comment':
                table_comment = row.data_type
                break
        
        # Get column details
        columns = []
        in_columns_section = True
        for row in props_df.collect():
            # Skip metadata rows (they start with # or are empty)
            if row.col_name.startswith('#') or row.col_name == '' or row.col_name in ['Owner', 'Created Time', 'Last Access', 'Created By', 'Type', 'Provider', 'Comment', 'Location', 'Serde Library', 'InputFormat', 'OutputFormat', 'Storage Properties', 'Partition Provider', 'Table Properties']:
                in_columns_section = False
                continue
            
            if in_columns_section and row.data_type and not row.data_type.startswith('['):
                columns.append({
                    'name': row.col_name,
                    'type': row.data_type,
                    'comment': row.comment if hasattr(row, 'comment') else None
                })
        
        return {
            'schema': schema_name,
            'table': table_name,
            'full_name': full_table,
            'table_comment': table_comment,
            'columns': columns
        }
    except Exception as e:
        print(f"⚠ Could not describe {full_table}: {e}")
        return None

# Collect metadata for all tables
all_tables_metadata = []

print("\n🔍 Collecting table metadata...")

for table in bronze_tables:
    meta = get_table_metadata(bronze_schema, table)
    if meta:
        all_tables_metadata.append(meta)
        print(f"  ✓ {bronze_schema}.{table}: {len(meta['columns'])} columns")

for table in gold_tables:
    meta = get_table_metadata(gold_schema, table)
    if meta:
        all_tables_metadata.append(meta)
        print(f"  ✓ {gold_schema}.{table}: {len(meta['columns'])} columns")

print(f"\n✅ Total: {len(all_tables_metadata)} tables")


In [ ]:
# Cell 6: Define DTA-Specific Column Comment Templates

# These provide rich descriptions for Genie when existing comments are missing
COLUMN_COMMENT_TEMPLATES = {
    # Common columns across tables
    'dta_id': 'Unique identifier (UUID) for the DTA. Foreign key to gold_md.dta.dta_id.',
    'trial_id': 'Clinical trial/study identifier (e.g., VAC18193RSV3001). Use for filtering DTAs by trial.',
    'data_stream_type': 'Category of clinical data (e.g., Labs, ECG, Adverse Events, Biomarkers). Use for filtering DTAs by data type.',
    'data_provider_name': 'External vendor/data provider name (e.g., LabCorp, Covance, PPD). Use for filtering DTAs by vendor.',
    'status': 'Lifecycle status: DRAFT, ACTIVE, MANUAL_REVIEW, or ARCHIVED.',
    'workflow_state': 'Approval workflow state: NOT_STARTED, IN_REVIEW, APPROVED, or REJECTED.',
    'version': 'Version identifier linking to md_version_registry.',
    'is_current': 'TRUE for the currently active version. Use to filter for latest records.',
    'domain_info': 'Excel sheet name or data domain from which the record was extracted.',
    'vendor_comment': 'Comments added by the vendor during DTA review.',
    'definition_hash': 'SHA-256 hash for deduplication. Identical definitions share the same hash.',
    
    # DTA-specific
    'dta_number': 'Human-readable sequential DTA identifier (e.g., DTA001, DTA002).',
    'dta_name': 'User-friendly name for the DTA. Format: {trial_id}_{data_stream_type}_{vendor}.',
    'latest_major_version': 'Most recent approved version in Gold library. NULL if never approved.',
    'current_draft_version': 'Active draft version in Silver layer.',
    
    # Workflow
    'workflow_iteration': 'Approval cycle number. First submission is 1, resubmission is 2, etc.',
    'workflow_status': 'Status of this workflow iteration: NOT_STARTED, IN_REVIEW, APPROVED, REJECTED.',
    'approval_status': 'Individual approver status: PENDING, APPROVED, REJECTED.',
    'approver_role': 'Role of the approver: jnj_dae, vendor, or librarian.',
    'approver_principal': 'Email or identity of the assigned approver.',
    
    # Version Registry
    'version_type': 'Version category: DTA_TEMPLATE (canonical), DTA_APPROVED (approved), DTA_DRAFT (work in progress).',
    'library_type': 'Metadata library type: transfer_variables, test_concepts, codelists, etc.',
    'is_latest_major': 'TRUE if this is the most recent Library Major version.',
    'record_count': 'Number of records in this version.',
    
    # Transfer Variables
    'transfer_variable_name': 'Technical name of the data element (e.g., STUDYID, SUBJID).',
    'transfer_variable_order': 'Sequence number for display and processing order.',
    'format': 'Data type specification: TEXT, NUMERIC, DATE, DATETIME.',
    'anticipated_max_length': 'Maximum character length for the field.',
    'transfer_file_key': 'TRUE if this variable is part of the unique record key.',
    'populate_for_all_records': 'TRUE if required for all records, FALSE if optional.',
    
    # Test Concepts
    'test_concept_id': 'Unique identifier (UUID) for this test concept record.',
    'test_concept_reference': 'Unique reference for the test concept (e.g., LBTESTCD value).',
    'transfer_tuple_map': 'MAP<STRING, STRING> storing all column-value pairs from source Excel row.',
    
    # Activity Log
    'activity_id': 'Unique identifier for this activity record.',
    'activity_category': 'High-level category: DATA_CHANGE, WORKFLOW, STATUS_CHANGE, VERSION.',
    'activity_type': 'Specific activity: INSERT, UPDATE, DELETE, APPROVED, REJECTED, etc.',
    'performed_by_principal': 'User who performed the action.',
    'performed_ts': 'Timestamp when the activity occurred.',
    
    # Audit columns
    'created_by_principal': 'User who created the record.',
    'created_ts': 'Timestamp when the record was created.',
    'last_updated_by_principal': 'User who last updated the record.',
    'last_updated_ts': 'Timestamp of last update.',
}

# DTA-specific table descriptions
TABLE_COMMENT_TEMPLATES = {
    'dta': 'Core DTA (Data Transfer Agreement) entity. Search by data_provider_name, data_stream_type, or trial_id. Join with dta_workflow for approval status.',
    'dta_workflow': 'Approval workflow tracking. Each DTA can have multiple workflow iterations.',
    'dta_approval_task': 'Individual approval tasks within a workflow. Includes JNJ DAE, vendor, and librarian approvals.',
    'dta_activity_log': 'Full audit history of all DTA activities. Query by dta_id and performed_by_principal.',
    'md_version_registry': 'Central registry for all DTA versions. Filter by version_type and dta_id.',
    'md_dta_transfer_variables': 'Approved transfer variable library. Filter by is_current=true for latest.',
    'md_dta_vendor_test_concepts': 'Approved test concept library. Filter by is_current=true for latest.',
    'md_file_history': 'Document manifest tracking file processing status.',
    'md_document_types': 'Reference table for enabled document types.',
    'md_dta_excel_file_raw': 'Excel sheet metadata from tsDTA files.',
}

print(f"✓ Loaded {len(COLUMN_COMMENT_TEMPLATES)} column templates")
print(f"✓ Loaded {len(TABLE_COMMENT_TEMPLATES)} table templates")


In [ ]:
# Cell 7: Generate add_table_comments.sql

def generate_table_comments_sql(tables_metadata):
    """Generate SQL statements for table and column comments."""
    lines = [
        "-- ============================================================================",
        "-- Table and Column Comments for Databricks Genie AI Assistant",
        "-- ============================================================================",
        f"-- Generated: {datetime.now().isoformat()}",
        f"-- Catalog: {catalog}",
        "--",
        "-- Purpose: Add comprehensive descriptions to enable Genie to answer",
        "--          natural language questions about DTAs and clinical metadata.",
        "--",
        "-- Usage: Run this script after table creation or schema changes.",
        "-- ============================================================================",
        "",
    ]
    
    current_schema = None
    
    for meta in sorted(tables_metadata, key=lambda x: (x['schema'], x['table'])):
        schema = meta['schema']
        table = meta['table']
        full_name = f"{schema}.{table}"
        
        # Add schema header
        if schema != current_schema:
            current_schema = schema
            layer = "BRONZE" if schema == bronze_schema else "GOLD" if schema == gold_schema else "SILVER"
            lines.extend([
                "",
                f"-- ============================================================================",
                f"-- {layer} LAYER: {schema}",
                f"-- ============================================================================",
                "",
            ])
        
        # Table comment
        table_comment = meta.get('table_comment') or TABLE_COMMENT_TEMPLATES.get(table, f'Table {table} in {schema} schema.')
        # Escape single quotes
        table_comment = table_comment.replace("'", "''")
        
        lines.extend([
            f"-- Table: {full_name}",
            f"COMMENT ON TABLE {full_name} IS '{table_comment}';",
            "",
        ])
        
        # Column comments
        for col in meta['columns']:
            col_name = col['name']
            # Use existing comment or template or generate default
            col_comment = col.get('comment') or COLUMN_COMMENT_TEMPLATES.get(col_name, f"Column {col_name} of type {col['type']}.")
            col_comment = col_comment.replace("'", "''")
            
            lines.append(f"ALTER TABLE {full_name} ALTER COLUMN {col_name} COMMENT '{col_comment}';")
        
        lines.append("")
    
    # Verification query
    lines.extend([
        "",
        "-- ============================================================================",
        "-- Verification",
        "-- ============================================================================",
        f"SELECT '✅ Comments added for {len(tables_metadata)} tables' as status;",
    ])
    
    return "\n".join(lines)

# Generate the SQL
table_comments_sql = generate_table_comments_sql(all_tables_metadata)

# Save to file
comments_path = f"{sql_dir}/add_table_comments.sql"
with open(comments_path, 'w') as f:
    f.write(table_comments_sql)

print(f"✓ Generated: {comments_path}")
print(f"  Size: {len(table_comments_sql):,} characters")


In [ ]:
# Cell 8: Generate genie_instructions.txt

genie_instructions = f"""# DTA Genie Space Instructions

## Purpose
Help JNJ Data Acquisition Experts (DAEs) and Vendors search for, compare, and create Data Transfer Agreements (DTAs) based on approved versions or templates.

## Required Search Criteria - ALWAYS ASK FOR CLARIFICATION

When users ask about DTAs but don't specify required filters, you MUST ask:
"To find relevant DTAs, please specify:
- The vendor name (data_provider_name), AND
- Either the data stream type (e.g., Labs, ECG, Biomarkers) OR trial ID (e.g., VAC18193RSV3001)"

Required filter combinations:
1. data_provider_name + data_stream_type
2. data_provider_name + trial_id

## Key Business Terms

| Term | Meaning | SQL Filter |
|------|---------|------------|
| Active DTA | Approved and in production | status = 'ACTIVE' AND workflow_state = 'APPROVED' |
| Draft DTA | Work in progress | status = 'DRAFT' |
| Pending Approval | Awaiting reviewer action | workflow_state = 'IN_REVIEW' |
| Latest Version | Most recent approved | is_current = true |
| DTA Template | Canonical template | version_type = 'DTA_TEMPLATE' |

## DTA Workflow States

- **NOT_STARTED**: Draft created, not yet submitted for approval
- **IN_REVIEW**: Submitted, awaiting approvals from JNJ DAE, Vendor, or Librarian
- **APPROVED**: All required approvals complete
- **REJECTED**: Returned for revision with comments

## Approver Roles

- **jnj_dae**: J&J Data Acquisition Expert - internal reviewer
- **vendor**: External data provider representative
- **librarian**: J&J Metadata Librarian - for template promotion

## Common Query Patterns

### Find DTAs by vendor
Filter: data_provider_name LIKE '%{{vendor}}%'

### Find DTAs by trial
Filter: trial_id LIKE '%{{trial}}%'

### Find DTAs by data stream
Filter: data_stream_type = '{{stream}}'

### Latest N versions
ORDER BY created_ts DESC LIMIT N

### DTAs modified by user
Join dta_activity_log and filter: performed_by_principal LIKE '%{{email}}%'

### Similar DTAs for cloning
Match on data_stream_type with similar data_provider_name or trial_id

## Data Stream Types

Common values: Labs, ECG, Adverse Events, Biomarkers, Demographics, Vital Signs, PF (Patient Flow), Medical History

## Table Relationships

```
dta (core entity)
  |-- dta_workflow (1:N by dta_id)
  |     |-- dta_approval_task (1:N by dta_workflow_id)
  |-- md_version_registry (1:N by dta_id)
  |-- md_dta_transfer_variables (1:N by dta_id, filter is_current=true)
  |-- md_dta_vendor_test_concepts (1:N by dta_id, filter is_current=true)
  |-- dta_activity_log (1:N by dta_id)
```

## Important Notes

1. Always filter transfer_variables and test_concepts by is_current = true for latest versions
2. Use UPPER() or ILIKE for case-insensitive matching on vendor/trial names
3. DTA numbers are sequential: DTA001, DTA002, etc.
4. Version format: 1.0-DTA001-v1.0 (approved) or 1.0-DTA001-draft1 (draft)

---
Generated: {datetime.now().isoformat()}
"""

# Save to file
instructions_path = f"{export_base}/genie_instructions.txt"
with open(instructions_path, 'w') as f:
    f.write(genie_instructions)

print(f"✓ Generated: {instructions_path}")
print(f"  Size: {len(genie_instructions):,} characters")


In [ ]:
# Cell 9: Generate example_queries.sql

example_queries_sql = f"""-- ============================================================================
-- Example SQL Queries for Genie Training
-- ============================================================================
-- Generated: {datetime.now().isoformat()}
-- Catalog: {catalog}
--
-- Format: Each query has a QUESTION (natural language) and SQL answer.
-- Parameterized queries use :param syntax for trusted responses.
-- ============================================================================


-- ============================================================================
-- QUERY 1: Find DTAs by vendor and data stream (parameterized)
-- ============================================================================
-- QUESTION: Show me DTAs for vendor :vendor_name and data stream :data_stream

SELECT 
    dta_number,
    dta_name,
    trial_id,
    data_stream_type,
    data_provider_name,
    status,
    workflow_state,
    latest_major_version,
    last_updated_ts
FROM {gold_schema}.dta
WHERE UPPER(data_provider_name) LIKE UPPER(CONCAT('%', :vendor_name, '%'))
  AND UPPER(data_stream_type) LIKE UPPER(CONCAT('%', :data_stream, '%'))
ORDER BY last_updated_ts DESC;


-- ============================================================================
-- QUERY 2: Find DTAs for a specific trial
-- ============================================================================
-- QUESTION: Show all DTAs for trial :trial_id

SELECT 
    dta_number,
    dta_name,
    data_stream_type,
    data_provider_name,
    status,
    workflow_state,
    latest_major_version
FROM {gold_schema}.dta
WHERE UPPER(trial_id) LIKE UPPER(CONCAT('%', :trial_id, '%'))
ORDER BY data_stream_type, data_provider_name;


-- ============================================================================
-- QUERY 3: Latest N versions for a DTA
-- ============================================================================
-- QUESTION: What are the latest 2 approved versions for DTA :dta_number?

SELECT 
    v.version,
    v.version_type,
    v.status,
    v.record_count,
    v.created_ts,
    v.created_by_principal
FROM {gold_schema}.md_version_registry v
JOIN {gold_schema}.dta d ON v.dta_id = d.dta_id
WHERE d.dta_number = :dta_number 
  AND v.version_type = 'DTA_APPROVED'
ORDER BY v.created_ts DESC 
LIMIT 2;


-- ============================================================================
-- QUERY 4: DTAs modified by a specific user
-- ============================================================================
-- QUESTION: Show DTAs created or modified by user :user_email

SELECT DISTINCT
    d.dta_number,
    d.dta_name,
    d.trial_id,
    d.data_stream_type,
    d.data_provider_name,
    d.status,
    d.workflow_state
FROM {gold_schema}.dta d
JOIN {gold_schema}.dta_activity_log a ON d.dta_id = a.dta_id
WHERE UPPER(a.performed_by_principal) LIKE UPPER(CONCAT('%', :user_email, '%'))
ORDER BY d.dta_number;


-- ============================================================================
-- QUERY 5: Approval status for a DTA
-- ============================================================================
-- QUESTION: What is the approval status for DTA :dta_number?

SELECT 
    d.dta_number,
    d.dta_name,
    w.workflow_status,
    w.workflow_iteration,
    t.approver_role,
    t.approver_principal,
    t.approval_status,
    t.approved_ts,
    t.approval_comment
FROM {gold_schema}.dta d
JOIN {gold_schema}.dta_workflow w ON d.dta_id = w.dta_id
JOIN {gold_schema}.dta_approval_task t ON w.dta_workflow_id = t.dta_workflow_id
WHERE d.dta_number = :dta_number
ORDER BY w.workflow_iteration DESC, t.approval_order;


-- ============================================================================
-- QUERY 6: Count transfer variables in a DTA
-- ============================================================================
-- QUESTION: How many transfer variables are in DTA :dta_number?

SELECT 
    d.dta_number,
    d.dta_name,
    COUNT(*) as variable_count
FROM {gold_schema}.md_dta_transfer_variables tv
JOIN {gold_schema}.dta d ON tv.dta_id = d.dta_id
WHERE d.dta_number = :dta_number 
  AND tv.is_current = true
GROUP BY d.dta_number, d.dta_name;


-- ============================================================================
-- QUERY 7: List all approved DTA templates
-- ============================================================================
-- QUESTION: List all approved DTA templates

SELECT 
    version,
    library_type,
    record_count,
    status,
    created_ts,
    created_by_principal
FROM {gold_schema}.md_version_registry
WHERE version_type = 'DTA_TEMPLATE' 
  AND status = 'ACTIVE'
ORDER BY library_type, created_ts DESC;


-- ============================================================================
-- QUERY 8: DTAs pending approval
-- ============================================================================
-- QUESTION: Which DTAs are pending approval?

SELECT 
    d.dta_number,
    d.dta_name,
    d.trial_id,
    d.data_provider_name,
    d.data_stream_type,
    w.workflow_status,
    w.initiated_ts
FROM {gold_schema}.dta d
JOIN {gold_schema}.dta_workflow w ON d.dta_id = w.dta_id
WHERE d.workflow_state = 'IN_REVIEW' 
  AND w.workflow_status = 'IN_REVIEW'
ORDER BY w.initiated_ts;


-- ============================================================================
-- QUERY 9: Recent activity for a vendor
-- ============================================================================
-- QUESTION: Show recent DTA activity for vendor :vendor_name

SELECT 
    d.dta_number,
    d.dta_name,
    a.activity_type,
    a.activity_summary,
    a.performed_by_principal,
    a.performed_ts
FROM {gold_schema}.dta_activity_log a
JOIN {gold_schema}.dta d ON a.dta_id = d.dta_id
WHERE UPPER(d.data_provider_name) LIKE UPPER(CONCAT('%', :vendor_name, '%'))
ORDER BY a.performed_ts DESC 
LIMIT 20;


-- ============================================================================
-- QUERY 10: Transfer variables for a DTA
-- ============================================================================
-- QUESTION: Show transfer variables for DTA :dta_number

SELECT 
    tv.transfer_variable_order,
    tv.transfer_variable_name,
    tv.format,
    tv.anticipated_max_length,
    tv.transfer_file_key,
    tv.populate_for_all_records,
    tv.domain_info,
    tv.variable_description
FROM {gold_schema}.md_dta_transfer_variables tv
JOIN {gold_schema}.dta d ON tv.dta_id = d.dta_id
WHERE d.dta_number = :dta_number 
  AND tv.is_current = true
ORDER BY tv.transfer_variable_order;


-- ============================================================================
-- QUERY 11: Find similar DTAs for cloning
-- ============================================================================
-- QUESTION: Find DTAs similar to :dta_number that I can clone

WITH source_dta AS (
    SELECT data_stream_type, data_provider_name
    FROM {gold_schema}.dta
    WHERE dta_number = :dta_number
)
SELECT 
    d.dta_number,
    d.dta_name,
    d.trial_id,
    d.data_stream_type,
    d.data_provider_name,
    d.status,
    d.latest_major_version
FROM {gold_schema}.dta d, source_dta s
WHERE d.dta_number != :dta_number
  AND (d.data_stream_type = s.data_stream_type 
       OR d.data_provider_name = s.data_provider_name)
  AND d.workflow_state = 'APPROVED'
ORDER BY 
    CASE WHEN d.data_stream_type = s.data_stream_type AND d.data_provider_name = s.data_provider_name THEN 1
         WHEN d.data_stream_type = s.data_stream_type THEN 2
         ELSE 3 END,
    d.last_updated_ts DESC;


-- ============================================================================
-- QUERY 12: Draft DTAs in progress
-- ============================================================================
-- QUESTION: Show all draft DTAs

SELECT 
    dta_number,
    dta_name,
    trial_id,
    data_stream_type,
    data_provider_name,
    current_draft_version,
    created_by_principal,
    last_updated_ts
FROM {gold_schema}.dta
WHERE status = 'DRAFT'
ORDER BY last_updated_ts DESC;


-- ============================================================================
-- QUERY 13: Active approved DTAs
-- ============================================================================
-- QUESTION: Show all active approved DTAs

SELECT 
    dta_number,
    dta_name,
    trial_id,
    data_stream_type,
    data_provider_name,
    latest_major_version,
    last_updated_ts
FROM {gold_schema}.dta
WHERE status = 'ACTIVE' 
  AND workflow_state = 'APPROVED'
ORDER BY last_updated_ts DESC;


-- ============================================================================
-- QUERY 14: DTAs updated recently
-- ============================================================================
-- QUESTION: What DTAs were updated in the last 30 days?

SELECT 
    dta_number,
    dta_name,
    trial_id,
    data_stream_type,
    data_provider_name,
    status,
    workflow_state,
    last_updated_ts
FROM {gold_schema}.dta
WHERE last_updated_ts >= DATE_ADD(CURRENT_DATE(), -30)
ORDER BY last_updated_ts DESC
LIMIT 20;


-- ============================================================================
-- QUERY 15: Test concepts for a DTA
-- ============================================================================
-- QUESTION: Show test concepts for DTA :dta_number

SELECT 
    tc.test_concept_reference,
    tc.domain_info,
    tc.status,
    tc.vendor_comment
FROM {gold_schema}.md_dta_vendor_test_concepts tc
JOIN {gold_schema}.dta d ON tc.dta_id = d.dta_id
WHERE d.dta_number = :dta_number 
  AND tc.is_current = true
ORDER BY tc.test_concept_reference;


-- ============================================================================
-- End of Example Queries
-- ============================================================================
"""

# Save to file
queries_path = f"{sql_dir}/example_queries.sql"
with open(queries_path, 'w') as f:
    f.write(example_queries_sql)

print(f"✓ Generated: {queries_path}")
print(f"  Size: {len(example_queries_sql):,} characters")
print(f"  Queries: 15 examples")


In [ ]:
# Cell 10: Create Export Manifest and Set Task Values

manifest = {
    "export_type": "genie_assets",
    "export_timestamp": datetime.now().isoformat(),
    "catalog": catalog,
    "source_root": source_root,
    "files": [
        {
            "filename": "sql/add_table_comments.sql",
            "description": "Table and column comments for Genie",
            "tables_count": len(all_tables_metadata)
        },
        {
            "filename": "genie_instructions.txt",
            "description": "DTA-specific Genie space instructions"
        },
        {
            "filename": "sql/example_queries.sql",
            "description": "Parameterized SQL queries for Genie training",
            "queries_count": 15
        }
    ],
    "tables_discovered": {
        "bronze": bronze_tables,
        "gold": gold_tables
    }
}

manifest_path = f"{export_base}/export_manifest.json"
with open(manifest_path, 'w') as f:
    json.dump(manifest, f, indent=2, default=str)

print(f"✓ Created manifest: {manifest_path}")

# Set Task Values for Downstream
result = {
    "success": True,
    "export_type": "genie_assets",
    "export_base": export_base,
    "files_created": 3,
    "tables_discovered": len(all_tables_metadata),
    "export_timestamp": manifest["export_timestamp"]
}

dbutils.jobs.taskValues.set(key="export_result", value=json.dumps(result))

print("\n" + "="*60)
print("GENIE ASSETS EXPORT COMPLETE")
print("="*60)
print(f"Export Path: {export_base}")
print(f"Tables Discovered: {len(all_tables_metadata)}")
print(f"Files Created:")
print(f"  - sql/add_table_comments.sql")
print(f"  - genie_instructions.txt")
print(f"  - sql/example_queries.sql")
print(f"  - export_manifest.json")
print("="*60)
